# 처음 배우는 셸 스크립트#7

## 3부 예제와 함께 활용하는 셸 스크립트 활용
시스템 운영, 개발환경 구축, 클라우드 시스템 운영 시 발생하는 다양한 상황에서 셸 스크립트를 활용한다.  
상황 -> 방법 찾기 -> 스크립트 생성 -> 문제 해결  

## 10 보안
리눅스 운영체제의 보안을 셸 스크립트로 제어한다

### 10.1 패스워드 생성 법칙을 적용할 때
##### 상황 
패스워드를 복잡하게 설정하여 외부로부터 침입하기 어려운 패스워드만 만들게 한다  
pam_pwquality라는 라이브러리로 설정할 수 있다
##### 방법 찾기
페도라 계열은 pam_pwquality가 기본 탑재이지만 데비안 계열은 별도 설치해야 한다    
필요한 정보  
- 페도라 계열의 리눅스 환경 설정파일 경로 : /etc/pam.d/system-auth
- 데비안 계열의 리눅스 환경 설정파일 경로: /etc/pam.d/common-password
- Retry: 패스워드 입력 실패 시 재시도 횟수
- Minlen: 최소 패스워드 길이
- Difok: 이전 비밀번호와 유사한 문자 개수
- Lcredit: 소문자 최소 요구 개수
- Ucredit: 대문자 최소 요구 개수
- Ocredit: 특수 문자 최소 요구 개수
- Enforce_for_root: root 사용자 패스워드 생성 법직 적용
프로세스
- 운영체제 타입을 확인한다
- 페도라 계열 리눅스면 /etc/pam.d/system-auth 파일에 설정을 적용한다
- 데비안 계열 리눅스면 /etc/pam.d/common-password 파일이 있는지 확인한다
- 파일이 없으면 libpam-pwquality 패키지를 설치한다
- 파일이 있으면 /etc/pam.d/common-password에 설정을 적용한다
##### 스크립트 생성
레드햇/CentOS 8버전, 우분투 18.04 LTS 이상을 기준으로 작성했다

In [ ]:
%%bash
#!/bin/bash

# 운영체제 타입 확인
ostype=$(cat /etc/*release| grep ID_LIKE | sed "s/ID_LIKE=//;s/\"//g")

# 운영체제가 페도라 계열일 경우
if [[ $ostype == "fedora" ]]; then
  # 설정 여부 체크
  conf_chk=$(cat /etc/pam.d/system-auth | grep 'local_users_only$' | wc -l)
  # 설정이 안되어 있으면 설정 후 설정 내용 확인
  # \1에 대한 내용 https://stackoverflow.com/questions/4609949/what-does-1-in-sed-do 
  # \1 refers to the characters captured by the escaped parentheses.
  # Note: it is not needed to espace parentheses with a backslash if you use extended regex with sed -E

  if [ $conf_chk -eq 1 ]; then
    sed -i 's/\(local_users_only$\)/\1 retry=3 authtok_type= minlen=8 lcredit=-1 ucredit=-1 dcredit=-1 ocredit=-1 enforce_for_root/g' /etc/pam.d/system-auth
    cat /etc/pam.d/system-auth | grep '^password[[:space:]]*requisite'
  fi
# 운영체제가 데비안 계열일 경우
elif [[ $ostype == "debian" ]]; then
  # pam_pwquality.so가 설치되어 있는지 설정파일을 통해 확인
  conf_chk=$(cat /etc/pam.d/common-password | grep 'pam_pwquality.so' | wc -l)
  # 설치가 안되어 있으면 libpam-pwquality 설치
  if [ $conf_chk -eq 0 ]; then
     apt install libpam-pwquality
  fi
  # 설정 여부 체크
  conf_chk=$(cat /etc/pam.d/common-password | grep 'retry=3$' | wc -l)
  # 설정이 안되어 있으면 설정 후 설정 내용 확인
  if [ $conf_chk -eq 1 ]; then
     sed -i 's/\(retry=3$\)/\1 minlen=8 maxrepeat=3 ucredit=-1 lcredit=-1 dcredit=-1 ocredit=-1 difok=3 gecoscheck=1 reject_username enforce_for_root/g' /etc/pam.d/common-password
     echo "==========================================="
     cat /etc/pam.d/common-password | grep '^password[[:space:]]*requisite'
  fi
fi


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash conf-pwpolicy.sh


### 10.2 패스워드 변경 주기를 설정할 때
##### 상황 
패스워드를 복잡하게 강제한다음 변경 주기를 설정하여 3~6개월에 한 번씩 변경하도록 권고한다
##### 방법 찾기
chage라는 명령어로 설정할 수 있다   
필요한 정보  
- -d, --lastday LAST_DAY: 마지막으로 패스워드를 변경한 날짜 설정
- -E, --expiredate EXPIRE_DATE: 특정 계정의 패스워드 만료일 설정
- -l, --list: 패스워드 설정 주기 정보 확인
- -m, --mindays MIN_DAYS: 패스워드 변경 최소 설정일
- -M, --maxdays MAX_DAYS: 패스워드 변경 최대 설정일
- -W, --warndays WARN_DAYS: 패스워드 만료 경고일
프로세스
- 패스워드 설정 주기를 설정할 대상 서버 정보를 변수에 저장한다
- 패스워드 설정 주기를 설정할 사용자 계정을 변수에 저장한다
- for문을 돌면서 다음 프로세스를 수행한다
    - 패스워드 설정 주기가 설정되어 있는지 chage -l로 확인한다
    - 설정되어 있지 않다면, 90일로 설정한다
    - 설정 정보를 확인한다


##### 스크립트 생성

In [ ]:
#!/bin/bash

# 대상서버와 계정정보 변수 저장
hosts="host01 host02"
account="root stack user01 user02"

# 대상서버만큼 반복
for host in $hosts; do
  echo "###### $host ######"
  # 계정정보만큼 반복
  for user in $account; do
    # 패스워드 설정 주기 체크
    pw_chk=$(ssh -q root@$host "chage -l $user | grep 99999 | wc -l") #Maximum number로 시작하는 항목의 값은 변경전 99999로 설정되어 있다.
    # 패스워드 설정 주기가 설정되어 있지 않다면
    if [[ $pw_chk -eq 1 ]]; then
      # 패스워드 설정 주기를 90일로 설정
      ssh -q root@$host "chage -d $(date +%Y-%m-%d) -M 90 $user"
      echo "======> $user"
      # 설정 결과 확인
      ssh -q root@$host "chage -l $user"
    fi
  done
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash conf-pwage.sh


### 10.3 디렉터리 및 파일 접근 권한 변경할 때
##### 상황 
리눅스는 파일 소유자나 그룹 소유자만 해당 파일을 읽고 쓰고 삭제할 수 있는 권한을 줄 수 있다  
이를 sticky bit라고 부르며, sticky bit가 파일이나 디렉터리 소유자에게 부여한 것을 SUID, 그룹 소유자에게 부여한 것을 SGID, 기타 사용자에게 부여한 것을 Sticky bit라 부른다.  
그런데 이런 파일들은 특정 명령어를 실행하여 root 권한 획득 및 서비스의 장애를 발생시킬 수 있다.  
따라서 불필요한 파일에 SUID, SGID, Sticky bit가 설정되어 있지 않도록 관리해야 한다.  
모든 사용자가 수정할 수 있는 World Writable 파일이 없어야 한다.  

##### 방법 찾기
Sticky bit가 적용된 파일과 World Writable 파일, 디렉터리를 찾아서 조치한다   
필요한 정보  
- SUID (Set User ID) 설정파일을 찾기 위한 명령어: find / -perm -04000
- SGID (Set Group ID) 설정파일을 찾기 위한 명령어: find / -perm -02000
- Sticky bit 설정파일을 찾기 위한 명령어: find / -perm -01000
- World Writable 파일이나 디렉터리를 찾기 위한 명령어: find / -xdev -perm -2
프로세스
- SUID, SGID, Sticky bit가 설정된 파일 및 디렉터리를 찾는다
- World Writable 파일, 디렉터리를 찾는다
- 찾은 파일 목록을 보여주고, 권한 변경 여부를 묻는다
- Y를 선택하면 Sticky bit 파일은 644로 권한을 변경한다
- World Writable 파일의 경우 기타 사용자 쓰기 권한을 제거한다
- 모든 파일의 권한 변경이 완료되면 결과를 보여준다
- N을 선택하면 스크립트를 그냥 종료한다
- 엔터키를 누르면 아무것도 입력하지 않았다는 메시지를 보여준 후 스크립트를 종료한다
- 이외에는 글자를 잘못 입력했다는 메시지를 보여준 후 스크립트를 종료한다

##### 스크립트 생성

In [ ]:
#!/bin/bash

# Sticky bit가 설정된 경로 검색
echo "=== SUID, SGID, Sticky bit Path ==="
s_file=$(find / -xdev -perm -04000 -o -perm -02000 -o -perm 01000 2>/dev/null | grep -e 'dump$' -e 'lp*-lpd$' -e 'newgrp$' -e 'restore$' -e 'at$' -e 'traceroute$')
find / -xdev -perm -04000 -o -perm -02000 -o -perm 01000 2>/dev/null | grep -e 'dump$' -e 'lp*-lpd$' -e 'newgrp$' -e 'restore$' -e 'at$' -e 'traceroute$' | xargs ls -dl

# World Writable 경로 검색
echo -e "\n=== World Writable Path ==="
w_file=$(find / -xdev -perm -2 -ls | grep -v 'l..........' | awk '{print $NF}')
find / -xdev -perm -2 -ls | grep -v 'l..........' | awk '{print $NF}' | xargs ls -dl

echo ""
read -p "Do you want to change file permission(y/n)? " result

if [[ $result == "y" ]]; then

  # Sticky bit 경로 권한 변경
  echo -e "\n=== Chmod SUID, SGID, Sticky bit Path ==="
  for file in $s_file; do
    echo "chmod -s $file"
    chmod -s $file
  done
  # Writable 경로 권한 변경
  echo -e "\n=== Chmod World Writable Path ==="
  for file in $w_file; do
    echo "chmod o-w $file"
    chmod o-w $file
  done

  # Sticky bit 경로 변경결과 조회
  echo -e "\n=== Result of Sticky bit Path ==="
  for file in $s_file; do
    ls -dl $file
  done
  # Writable 경로 변경결과 조회
  echo -e "\n=== Result of World Writable Path ==="
  for file in $w_file; do
    ls -dl $file
  done
# 파일권한 변경을 원하지 않을 경우
elif [[ $result == "n" ]]; then
  exit
# 파일권한 변경여부 질의에 아무것도 입력하지 않았을 경우 
elif [[ -z $result ]]; then
  echo "Yon didn't have any choice. Please check these files for security."
  exit
# 파일권한 변경여부 질의에 아무 글자나 입력했을 경우
else
  echo "You can choose only y or n."
  exit 
fi 


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash conf-file.sh


### 10.4 Firewall에 포트 추가할 때
##### 상황 
리눅스에는 자체적으로 외부로부터의 침입을 방지하기 위해 iptables나 firewalld와 같은 방화벽 서비스를 사용한다.  
애플리케이션을 설치할 때마다 해당 애플리케이션이 사용하는 포트를 iptables나 firewalld에 등록해야 한다.  
셸 스크립트로 추가해보자  

##### 방법 찾기
레드햇, CnetOS는 6버전 이전은 iptables를 사용했다. 7버전부터 firewalld로 변경되었다.  
데비안 계열은 ufw라는 방화벽을 사용한다.  
필요한 정보  
- firewalld에서 서비스 포트 추가 명령어: firewall-cmd --add-service=[service name]
- firewalld에서 포트 추가 명령어: firewall-cmd --add-port=[port/protocol]
- ufw에서 서비스 포트 추가 명령어: ufw allow [service name | port/protocol]

프로세스
- 운영체제 타입이 페도라 계열인지 데비안 계열인지 확인한다
- 운영체제가 페도라 계열이면 시스템에 firewalld가 실행중인지 확인한다
- 운영체제가 데비안 계열이면 시스템에 ufw가 실행중인지 확인한다
- 사용자로부터 추가할 포트 목록을 엽력받는다
- 운영체제가 페도라 계열이면 firewall-cmd 명령어를 이용하여 포트를 추가한다
- 운영체제가 데비안 계열이면 ufw 명령어를 이용하여 포트를 추가한다

##### 스크립트 생성

In [ ]:
#!/bin/bash

# 운영체제 타입 확인
ostype=$(cat /etc/*release| grep ID_LIKE | sed "s/ID_LIKE=//;s/\"//g")

read -p "Please input ports(ex: http 123/tcp 123/udp) : " ports

if [[ -z $ports ]]; then echo "You didn't input port. Please retry."; exit; fi

# 운영체제가 페도라 계열일 경우
if [[ $ostype == "fedora" ]]; then
  # firewalld 실행상태 체크
  run_chk=$( firewall-cmd --state )
  if [[ $run_chk == "running" ]]; then
    # 입력받은 port 만큼 반복
    for port in $ports; do
       # service port 인지 일반 port인지 체크
       chk_port=$(echo $port | grep '^[a-zA-Z]' | wc -l)
       # service port일 경우
       if [[ chk_port -eq 1 ]]; then
         firewall-cmd --add-service=$port
         firewall-cmd --add-service=$port --permanent
       # 일반 port 일 경우
       else
         firewall-cmd --add-port=$port
         firewall-cmd --add-port=$port --permanent
       fi
    done
    # port 추가 결과 확인
    firewall-cmd --list-all
  fi
# 운영체제가 데비안 계열일 경우
elif [[ $ostype == "debian" ]]; then
  # ufw 실행상태 체크
  run_chk=$( ufw status | grep ": active" | wc -l )
  if [[ $run_chk -eq 1 ]]; then
    # 입력받은 port만큼 반복
    for port in $ports; do
      ufw allow $port
    done
    # port 추가 결과 확인
    ufw status numbered
  fi
fi


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash conf-firewall.sh


### 10.5 사설 인증서 생성할 때
##### 상황 
http 보안을 위해 공인 인증서를 발급받아 사용하거나, 사설 인증서를 생성하여 사용한다.  
내부 서비스나 개발용은 사설 인증서를 사용한다. 생성 과정이 복잡한데, 스크립트화 해 놓자.    

##### 방법 찾기
openssl 명령어를 이용하여 CA 인증서와 클라이언트 인증서를 생성한다.  
운영체제에 따라 서명용 호스트를 초기화하는 방법, 클라이언트에 인증기관용 인증서를 추가하는 명령어만 차이가 있다.    
필요한 정보  
- RSA 개인키 생성 명령어: openssl genrsa
- 자체 서명된 Root CA 인증서 생성 명령어: openssl req
- 자체 CA 인증서와 클라이언트키를 이용하여 클라이언트 인증서 생성 명령어: openssl ca
- 명령어별 상세 옵션 설명 확인: man genrsa, man req, man ca

프로세스
- 자체 서명된 Root CA 인증서가 생성될 디렉터리를 생성한다
- CA 디렉터리에 빈 index.tt와 1000이 입력된 serial을 생성한다
- 인증 기관용(CA) 개인키와 인증서를 생성한다
- 클라이언트에 인증기관용 인증서를 추가한다
- 추가한 인증서가 믿을 수 있는 인증서라고 설정한다
- 서버에서 사용할 SSL/TLS 서버 키를 만든다
- 서버에서 사용할 인증요청서를 만든다
- 서버의 인증요청서를 이용하여 CA에서 인증서를 발급받는다

##### 스크립트 생성
데비안, 우분투 따로 만들었다

In [ ]:
%%bash
#!/bin/bash
## 페도라
# 서명용 호스트 초기화
echo "=========================="
echo " Initializing sining host "
echo "=========================="
touch /etc/pki/CA/index.txt
echo '1000' | tee /etc/pki/CA/serial

# 인증 기관용 인증서 생성
echo "=================================="
echo " Creating a certificate authority "
echo "=================================="
echo "---------------------"
echo " Generate rsa ca key "
echo "---------------------"
openssl genrsa -out ca.key.pem 4096 #개인키 만듬
echo "--------------------------"
echo " Generate rsa ca cert key "
echo "--------------------------"
openssl req -key ca.key.pem -new -x509 -days 7300 -extensions v3_ca -out ca.crt.pem  #자체서명 인증서 만듬

# 클라이언트에 인증기관용 인증서 추가
echo "============================================="
echo " Adding the certificate authority to clients "
echo "============================================="
echo "cp ca.crt.pem /etc/pki/ca-trust/source/anchors/"
cp ca.crt.pem /etc/pki/ca-trust/source/anchors/
echo "update-ca-trust extract"
update-ca-trust extract

# SSL/TLS 키 생성
echo "========================="
echo " Creating an SSL/TLS key "
echo "========================="
openssl genrsa -out server.key.pem 2048 #클라이언트 개인키 만듬

# SSL/TLS 인증서 서명 요청용 키 생성
echo "================================================="
echo " Creating an SSL/TLS certificate signing request "
echo "================================================="
cp /etc/pki/tls/openssl.cnf .
openssl req -config openssl.cnf -key server.key.pem -new -out server.csr.pem #인증서 서명 요청

# SSL/TLS 인증서 생성
echo "=================================="
echo " Creating the SSL/TLS certificate "
echo "=================================="
openssl ca -config openssl.cnf -extensions v3_req -days 3650 -in server.csr.pem -out server.crt.pem -cert ca.crt.pem -keyfile ca.key.pem #서명해줌


In [ ]:
%%bash
#!/bin/bash
## 데비안
# 서명용 호스트 초기화
echo "=========================="
echo " Initializing sining host "
echo "=========================="
mkdir -p ./demoCA
mkdir -p ./demoCA/certs ./demoCA/crl ./demoCA/newcerts ./demoCA/private
touch ./demoCA/index.txt
echo '1000' | tee ./demoCA/serial

# 인증 기관용 인증서 생성
echo "=================================="
echo " Creating a certificate authority "
echo "=================================="
echo "---------------------"
echo " Generate rsa ca key "
echo "---------------------"
openssl genrsa -out ca.key.pem 4096
echo "--------------------------"
echo " Generate rsa ca cert key "
echo "--------------------------"
openssl req -key ca.key.pem -new -x509 -days 7300 -extensions v3_ca -out ca.crt.pem 

# 클라이언트에 인증기관용 인증서 추가
echo "============================================="
echo " Adding the certificate authority to clients "
echo "============================================="
echo "cp ca.crt.pem /usr/local/share/ca-certificates/"
cp ca.crt.pem /usr/local/share/ca-certificates/
echo "update-ca-certificates"
update-ca-certificates

# SSL/TLS 키 생성
echo "========================="
echo " Creating an SSL/TLS key "
echo "========================="
openssl genrsa -out server.key.pem 2048

# SSL/TLS 인증서 서명 요청용 키 생성
echo "================================================="
echo " Creating an SSL/TLS certificate signing request "
echo "================================================="
cp /usr/lib/ssl/openssl.cnf .
openssl req -config openssl.cnf -key server.key.pem -new -out server.csr.pem

# SSL/TLS 인증서 생성
echo "=================================="
echo " Creating the SSL/TLS certificate "
echo "=================================="
openssl ca -config openssl.cnf -extensions v3_req -days 3650 -in server.csr.pem -out server.crt.pem -cert ca.crt.pem -keyfile ca.key.pem


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
bash conf-certificate.sh